![alt text](https://drive.google.com/uc?export=view&id=1UXScsVx_Wni_JuDdB8LeTnM6jsPfIwkW)

Proprietary content. © Great Learning. All Rights Reserved. Unauthorized use or distribution prohibited.

# Sentiment Classification

### Dataset
- Dataset of 50,000 movie reviews from IMDB, labeled by sentiment positive (1) or negative (0)
- Reviews have been preprocessed, and each review is encoded as a sequence of word indexes (integers).
- For convenience, words are indexed by overall frequency in the dataset, so that for instance the integer "3" encodes the 3rd most frequent word in the data. This allows for quick filtering operations such as: "only consider the top 10,000 most common words, but eliminate the top 20 most common words".
- As a convention, "0" does not stand for a specific word, but instead is used to encode any unknown word.

Command to import data
- `from tensorflow.keras.datasets import imdb`

### Import the data (4 Marks)
- Use `imdb.load_data()` method
- Get train and test set
- Take 10000 most frequent words

In [1]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import tensorflow as tf
import warnings
warnings.filterwarnings('ignore')

from keras import models, regularizers, layers, optimizers, losses, metrics
from keras.models import Sequential
from keras.preprocessing import sequence  
from keras.layers import Dense, Flatten, Embedding, LSTM, TimeDistributed
from keras.optimizers import Adam
from keras.datasets import imdb

In [2]:
(X_train, y_train), (X_test, y_test) = imdb.load_data(path='imdb.npz', num_words=10000, skip_top=0, maxlen=None, seed=113,
    start_char=1, oov_char=2, index_from=3)

In [3]:
print(X_train.shape)
print(X_test.shape)

(25000,)
(25000,)


### Pad each sentence to be of same length (4 Marks)
- Take maximum sequence length as 300

In [4]:
X_train = sequence.pad_sequences(X_train, maxlen=300, dtype='int32', padding='pre', truncating='pre',value=0.0)
X_test =  sequence.pad_sequences(X_test, maxlen=300, dtype='int32', padding='pre', truncating='pre',value=0.0)

### Print shape of features & labels (4 Marks)

Number of review, number of words in each review

In [5]:
data = np.concatenate((X_train, X_test), axis=0)
targets = np.concatenate((y_train, y_test), axis=0)


In [6]:
print('Total Number of reviews :',data.shape[0])

Total Number of reviews : 50000


In [7]:
print('Number of words in each review: ',data.shape[1])

Number of words in each review:  300


Number of labels

In [8]:
print("Labels: ",np.unique(targets))

Labels:  [0 1]


In [9]:
print('Number of labels: ',targets.shape[0])

Number of labels:  50000


### Print value of any one feature and it's label (4 Marks)

Feature value

In [10]:
print(X_train[1])

[   0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    1
  194 1153  194 8255   78  228    5    6 1463 4369 5012  134   26    4
  715    8  118 1634   14  394   20   13  119  954  189  102    5  207
  110 3103   21   14   69  188    8   30   23    7    4  249  126   93
    4  114    9 2300 1523    5  647    4  116    9   35 8163    4  229
    9  340 1322    4  118    9    4  130 4901   19    4 1002    5   89
   29  952   46   37    4  455    9   45   43   38 1543 1905  398    4
 1649 

Label value

In [11]:
print(y_train[1])

0


### Decode the feature value to get original sentence (4 Marks)

First, retrieve a dictionary that contains mapping of words to their index in the IMDB dataset

In [12]:
word_index = imdb.get_word_index(path='imdb_word_index.json')


Now use the dictionary to get the original words from the encodings, for a particular sentence

In [13]:
reverse_word_index = dict([(value, key) for (key, value) in word_index.items()])

decoded_review = " ".join([reverse_word_index.get(i - 3, '#') for i in X_train[0]])

print(decoded_review)

# # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # this film was just brilliant casting location scenery story direction everyone's really suited the part they played and you could just imagine being there robert # is an amazing actor and now the same being director # father came from the same scottish island as myself so i loved the fact there was a real connection with this film the witty remarks throughout the film were great it was just brilliant so much that i bought the film as soon as it was released for # and would recommend it to everyone to watch and the fly fishing was amazing really cried at the end it was so sad and you know what they say if you cry at a film it must have been good and this definitely was also # to the two little boy's that played the # of norman and paul they were just brilliant children are often left out of the # list i think because the st

Get the sentiment for the above sentence
- positive (1)
- negative (0)

In [14]:
if y_train[0] == 1:
  print('Positive Review')
else:
  print('Negative Review')

Positive Review


### Define model (10 Marks)
- Define a Sequential Model
- Add Embedding layer
  - Embedding layer turns positive integers into dense vectors of fixed size
  - `tensorflow.keras` embedding layer doesn't require us to onehot encode our words, instead we have to give each word a unique integer number as an id. For the imdb dataset we've loaded this has already been done, but if this wasn't the case we could use sklearn LabelEncoder.
  - Size of the vocabulary will be 10000
  - Give dimension of the dense embedding as 100
  - Length of input sequences should be 300
- Add LSTM layer
  - Pass value in `return_sequences` as True
- Add a `TimeDistributed` layer with 100 Dense neurons
- Add Flatten layer
- Add Dense layer

In [15]:
model = Sequential()
model.add(Embedding(input_dim=10000,
                    output_dim=100,
                    input_length=300,
                    name='layer_embedding'))
model.add(LSTM(units=16, return_sequences=True))
model.add(TimeDistributed(Dense(100)))
model.add(Flatten())
model.add(Dense(1, activation='sigmoid'))


### Compile the model (4 Marks)
- Use Optimizer as Adam
- Use Binary Crossentropy as loss
- Use Accuracy as metrics

In [16]:
model.compile(loss='binary_crossentropy',
              optimizer=Adam(lr=1e-3),
              metrics=['accuracy'])

### Print model summary (4 Marks)

In [17]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
layer_embedding (Embedding)  (None, 300, 100)          1000000   
_________________________________________________________________
lstm (LSTM)                  (None, 300, 16)           7488      
_________________________________________________________________
time_distributed (TimeDistri (None, 300, 100)          1700      
_________________________________________________________________
flatten (Flatten)            (None, 30000)             0         
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 30001     
Total params: 1,039,189
Trainable params: 1,039,189
Non-trainable params: 0
_________________________________________________________________


### Fit the model (4 Marks)

In [18]:
%%time
model.fit(X_train, y_train, epochs=10, batch_size=64)

Epoch 1/10
391/391 [==============================] - 9s 23ms/step - loss: 0.3511 - accuracy: 0.8386
Epoch 2/10
391/391 [==============================] - 9s 22ms/step - loss: 0.1826 - accuracy: 0.9291
Epoch 3/10
391/391 [==============================] - 9s 22ms/step - loss: 0.0886 - accuracy: 0.9687
Epoch 4/10
391/391 [==============================] - 9s 22ms/step - loss: 0.0276 - accuracy: 0.9908
Epoch 5/10
391/391 [==============================] - 9s 22ms/step - loss: 0.0116 - accuracy: 0.9960
Epoch 6/10
391/391 [==============================] - 9s 22ms/step - loss: 0.0150 - accuracy: 0.9949
Epoch 7/10
391/391 [==============================] - 9s 24ms/step - loss: 0.0098 - accuracy: 0.9968
Epoch 8/10
391/391 [==============================] - 9s 23ms/step - loss: 0.0137 - accuracy: 0.9947
Epoch 9/10
391/391 [==============================] - 9s 22ms/step - loss: 0.0121 - accuracy: 0.9957
Epoch 10/10
391/391 [==============================] - 9s 22ms/step - loss: 0.0087 - accura

### Evaluate model (4 Marks)

In [19]:
%%time
result = model.evaluate(X_test, y_test)
print("Accuracy: {0:.2%}".format(result[1]))

782/782 [==============================] - 5s 7ms/step - loss: 1.0905 - accuracy: 0.8509
Accuracy: 85.09%
CPU times: user 5.89 s, sys: 306 ms, total: 6.19 s
Wall time: 5.7 s


### Predict on one sample (4 Marks)

In [20]:
y_pred = model.predict(x=X_test[100:])


In [21]:
if y_pred[1] > 0.5:
  print('Positive Review')
else:
  print('Negative Review')

Negative Review
